In [133]:
import os
import cv2
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import random
import magic
import matplotlib.pyplot as plt
from pathlib import Path
from os.path import exists

# Объединение детекций между yolov5 и yolov7 для класса human

In [253]:
def read_label(path):
    labels_car = []
    labels_human = []
    with open(path, 'r') as f:
        for line in f:
            data = line[:-1].split()
            if data[0] == '1':
                labels_human.append(line)
            elif data[0] == '0':
                labels_car.append(line)
    return labels_car, labels_human

def read_label_yolo(path):
    labels_car = []
    labels_human = []
    with open(path, 'r') as f:
        for line in f:
            data = line[:-1].split()
            if data[0] == '0':
                labels_human.append('1'+line[1:])
            elif data[0] == '2' or data[0] == '7':
                labels_car.append('0'+line[1:])
    return labels_car, labels_human

def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

def yolo2bb(box, dw, dh):
    class_id, x_center, y_center, w, h, conf = box.strip().split()
    x_center, y_center, w, h = float(x_center), float(y_center), float(w), float(h)
    x_center = round(x_center * dw)
    y_center = round(y_center * dh)
    w = round(w * dw)
    h = round(h * dh)
    x = round(x_center - w / 2)
    y = round(y_center - h / 2)

    return class_id, x, y, x+w, y+h, conf

def bb_to_yolo(box, image_w, image_h):
    x1, y1, x2, y2 = box
    return [((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), (x2 - x1)/image_w, (y2 - y1)/image_h]

def draw_label(class_id, box, image):
    
    colors = {0: (0,255,255), 1: (0,255,0), 2: (255,0,255)}
    x_l, y_t, x_r, y_b = box
    thickness = 1
    image = cv2.rectangle(image, (x_l, y_t), (x_r, y_b), colors[int(class_id)], thickness)

In [135]:
path_labelv5 = '../yolov5/runs/detect/exp_car_human_res/labels/'
path_labelv7 = '../yolov7/runs/detect/exp_car_human_res/labels/'
path_v7_res  = '../yolov7/runs/detect/exp_car_human_res_concat/'
path_images  = '../yolov7/runs/detect/exp_car_human/'

save_dir = Path(Path(path_v7_res) / 'labels')
save_dir.mkdir(parents=True, exist_ok=True)


for image_file in os.listdir(path_images):
    print(image_file)
    
#     if image_file != "23-11-2021_02-18-50_PM.jpg":
#             continue
    
    t = magic.from_file(path_images + image_file)
    w, h = t.split(', ')[-2].split('x')
    
    label_file = image_file[:-3] + 'txt'
    
    image = cv2.imread(path_images + image_file)
    
    labelv5_path = path_labelv5 + label_file
    labelv7_path = path_labelv7 + label_file
    
    existsv5 = exists(labelv5_path)
    existsv7 = exists(labelv7_path)
    
#     print(existsv5, existsv7)
    
    if not existsv5:
        shutil.copy(labelv7_path, labelv5_path)
    if not existsv7:
        shutil.copy(labelv5_path, labelv7_path)
    
    carv5, humanv5 = read_label(labelv5_path)
    carv7, humanv7 = read_label(labelv7_path)
#     print(humanv5)
#     print(humanv7)
    humanv5 = [yolo2bb(line, int(w), int(h)) for line in humanv5]
    humanv7 = [yolo2bb(line, int(w), int(h)) for line in humanv7]
    print("humanv5", humanv5)
    print("humanv5 len", len(humanv5))
    print("humanv7", humanv7)
    print("humanv7 len", len(humanv7))
    
    # Human processing
    human = []
    for detv7 in humanv7:
#         print(detv7)
        processed = False
        ious = []
#         print("humanv5 <<<<<", humanv5)
        for detv5 in humanv5:
            ious.append((bb_intersection_over_union(detv7[1:5], detv5[1:5]), detv5))
#         print("ious", ious)
        
        ioucount = [x for x in ious if x[0] > 0.50]
#         print(len(ioucount))
        if len(ioucount) > 1:
            for iou in ious:
                human.append(iou[1])
                humanv5.remove(iou[1])
                processed = True
        else:
            for iou in ious:
                if iou[0] > 0.80:
                    tofile = detv7 if detv7[5] > iou[1][5] else iou[1]
                    human.append(tofile)
                    humanv5.remove(iou[1])
                    processed = True
                elif iou[0] < 0.80 and iou[0] > 0.50 and \
                (detv7[3]-detv7[1])*(detv7[4]-detv7[2])*100/ (int(w)*int(h)) < 1:
                    tofile = detv7 if detv7[5] > iou[1][5] else iou[1]
                    human.append(tofile)
                    humanv5.remove(iou[1])
                    processed = True
                
        if not processed:
            if float(detv7[5]) > 0.55:
                    human.append(detv7) 
    
    if len(humanv5) != 0:
        for detv5 in humanv5:
            if float(detv5[5]) > 0.55:
                human.append(detv5)

                
    print("human", human)
    print("human len", len(human))

    with open(path_v7_res + '/labels/' + label_file, 'w') as f:
        for hum_lbl in human:
#             print(hum_lbl)
            
            box = ( int(hum_lbl[1]), int(hum_lbl[2]), int(hum_lbl[3]), int(hum_lbl[4])  )
            
            draw_label(hum_lbl[0], box, image)
            
            lbl = bb_to_yolo(box, int(w), int(h))
            data = hum_lbl[0] + ' ' + ' '.join(str(e) for e in lbl) + ' ' + hum_lbl[5] + '\n'
            f.write(data)
            
        for car_l in carv7:
#             print(car_lbl)
            
            car_lbl = yolo2bb(car_l, int(w), int(h))
            
            box = ( int(car_lbl[1]), int(car_lbl[2]), int(car_lbl[3]), int(car_lbl[4])  )
            
            draw_label(car_lbl[0], box, image)
            
#             lbl = bb_to_yolo(box, int(w), int(h))
#             data = hum_lbl[0] + ' ' + ' '.join(str(e) for e in lbl) + ' ' + hum_lbl[5] + '\n'
            
            f.write(str(car_l))
            
    cv2.imwrite(path_v7_res + '/' + image_file, image)
            
    
#     break

26-11-2021_08-22-27_AM.jpg
humanv5 [('1', 696, 89, 711, 127, '0.324991'), ('1', 458, 146, 481, 197, '0.708053')]
humanv5 len 2
humanv7 [('1', 458, 146, 479, 198, '0.875488')]
humanv7 len 1
human [('1', 458, 146, 479, 198, '0.875488')]
human len 1
25-11-2021_01-55-28_PM.jpg
humanv5 [('1', 1723, 244, 1743, 293, '0.348937'), ('1', 1029, 316, 1067, 408, '0.611594'), ('1', 1262, 256, 1288, 316, '0.648717'), ('1', 953, 600, 1083, 935, '0.774134')]
humanv5 len 4
humanv7 [('1', 1262, 265, 1285, 317, '0.63623'), ('1', 1030, 326, 1064, 404, '0.709961')]
humanv7 len 2
human [('1', 1262, 256, 1288, 316, '0.648717'), ('1', 1030, 326, 1064, 404, '0.709961'), ('1', 953, 600, 1083, 935, '0.774134')]
human len 3
25-11-2021_10-02-36_AM.jpg
humanv5 [('1', 386, 535, 450, 699, '0.871357')]
humanv5 len 1
humanv7 [('1', 383, 533, 449, 699, '0.940918')]
humanv7 len 1
human [('1', 383, 533, 449, 699, '0.940918')]
human len 1
23-11-2021_07-41-17_PM.jpg
humanv5 [('1', 412, 510, 456, 615, '0.473959'), ('1', 1464,

humanv5 [('1', 432, 452, 469, 526, '0.675292'), ('1', 1352, 345, 1398, 447, '0.877332')]
humanv5 len 2
humanv7 [('1', 1352, 346, 1399, 449, '0.897461')]
humanv7 len 1
human [('1', 1352, 346, 1399, 449, '0.897461'), ('1', 432, 452, 469, 526, '0.675292')]
human len 2
26-11-2021_10-09-10_AM.jpg
humanv5 [('1', 856, 196, 888, 284, '0.739255'), ('1', 768, 200, 825, 320, '0.790822')]
humanv5 len 2
humanv7 [('1', 768, 202, 816, 294, '0.731445'), ('1', 856, 201, 885, 273, '0.821777')]
humanv7 len 2
human [('1', 768, 200, 825, 320, '0.790822'), ('1', 856, 201, 885, 273, '0.821777')]
human len 2
25-11-2021_10-56-23_AM.jpg
humanv5 [('1', 1700, 387, 1734, 471, '0.810166')]
humanv5 len 1
humanv7 [('1', 1696, 386, 1736, 472, '0.861816')]
humanv7 len 1
human [('1', 1696, 386, 1736, 472, '0.861816')]
human len 1
25-11-2021_02-13-20_PM.jpg
humanv5 [('1', 1120, 896, 1180, 1017, '0.818527'), ('1', 1040, 874, 1127, 1050, '0.88824')]
humanv5 len 2
humanv7 [('1', 1120, 898, 1177, 1023, '0.694336'), ('1', 103

humanv5 [('1', 2001, 602, 2057, 763, '0.743878')]
humanv5 len 1
humanv7 [('1', 1996, 605, 2057, 757, '0.84668')]
humanv7 len 1
human [('1', 1996, 605, 2057, 757, '0.84668')]
human len 1
26-11-2021_04-11-29_AM.jpg
humanv5 [('1', 1000, 213, 1047, 325, '0.605184'), ('1', 573, 200, 609, 289, '0.70114')]
humanv5 len 2
humanv7 [('1', 276, 206, 295, 265, '0.275146'), ('1', 1278, 60, 1293, 95, '0.366699'), ('1', 1474, 71, 1495, 123, '0.600098'), ('1', 1500, 75, 1519, 123, '0.600586'), ('1', 999, 212, 1045, 333, '0.73877'), ('1', 571, 200, 607, 290, '0.857422')]
humanv7 len 6
human [('1', 1474, 71, 1495, 123, '0.600098'), ('1', 1500, 75, 1519, 123, '0.600586'), ('1', 999, 212, 1045, 333, '0.73877'), ('1', 571, 200, 607, 290, '0.857422')]
human len 4
24-11-2021_04-37-21_AM.jpg
humanv5 [('1', 638, 268, 711, 445, '0.757274')]
humanv5 len 1
humanv7 [('1', 1946, 189, 1985, 277, '0.389648'), ('1', 639, 279, 705, 447, '0.893066')]
humanv7 len 2
human [('1', 639, 279, 705, 447, '0.893066')]
human len 1

humanv5 [('1', 112, 591, 148, 679, '0.867184')]
humanv5 len 1
humanv7 [('1', 112, 592, 147, 683, '0.921387')]
humanv7 len 1
human [('1', 112, 592, 147, 683, '0.921387')]
human len 1
26-11-2021_07-46-02_AM.jpg
humanv5 []
humanv5 len 0
humanv7 []
humanv7 len 0
human []
human len 0
26-11-2021_03-55-17_AM.jpg
humanv5 [('1', 1442, 364, 1471, 440, '0.770824')]
humanv5 len 1
humanv7 [('1', 1438, 362, 1471, 441, '0.796875')]
humanv7 len 1
human [('1', 1438, 362, 1471, 441, '0.796875')]
human len 1
25-11-2021_04-04-52_AM.jpg
humanv5 [('1', 116, 626, 305, 847, '0.916905')]
humanv5 len 1
humanv7 [('1', 116, 622, 303, 850, '0.944336')]
humanv7 len 1
human [('1', 116, 622, 303, 850, '0.944336')]
human len 1
26-11-2021_03-37-22_AM.jpg
humanv5 [('1', 1048, 88, 1076, 155, '0.750707'), ('1', 1184, 126, 1217, 218, '0.774769')]
humanv5 len 2
humanv7 [('1', 1048, 90, 1075, 161, '0.866699'), ('1', 1185, 128, 1221, 220, '0.891602')]
humanv7 len 2
human [('1', 1048, 90, 1075, 161, '0.866699'), ('1', 1185, 12

24-11-2021_05-29-16_AM.jpg
humanv5 [('1', 1740, 22, 1828, 253, '0.870142')]
humanv5 len 1
humanv7 [('1', 1744, 22, 1832, 257, '0.888184')]
humanv7 len 1
human [('1', 1744, 22, 1832, 257, '0.888184')]
human len 1
26-11-2021_05-40-18_AM.jpg
humanv5 [('1', 908, 612, 944, 715, '0.795202')]
humanv5 len 1
humanv7 [('1', 910, 616, 944, 715, '0.836914')]
humanv7 len 1
human [('1', 910, 616, 944, 715, '0.836914')]
human len 1
26-11-2021_07-46-34_AM.jpg
humanv5 [('1', 1348, 222, 1377, 285, '0.805326')]
humanv5 len 1
humanv7 [('1', 974, 118, 1000, 167, '0.611328'), ('1', 1344, 221, 1377, 287, '0.880859')]
humanv7 len 2
human [('1', 974, 118, 1000, 167, '0.611328'), ('1', 1344, 221, 1377, 287, '0.880859')]
human len 2
25-11-2021_03-46-33_AM.jpg
humanv5 [('1', 411, 284, 465, 398, '0.762616'), ('1', 605, 8, 649, 124, '0.855681')]
humanv5 len 2
humanv7 [('1', 608, 10, 649, 122, '0.887695')]
humanv7 len 1
human [('1', 608, 10, 649, 122, '0.887695'), ('1', 411, 284, 465, 398, '0.762616')]
human len 2
2

...

humanv5 [('1', 1635, 216, 1649, 249, '0.442168'), ('1', 1312, 280, 1329, 323, '0.539732'), ('1', 1586, 234, 1603, 274, '0.54702'), ('1', 866, 412, 901, 467, '0.76393'), ('1', 1593, 444, 1625, 535, '0.774855'), ('1', 1642, 333, 1669, 401, '0.814354'), ('1', 1544, 432, 1584, 538, '0.838412'), ('1', 514, 530, 562, 635, '0.883384')]
humanv5 len 8
humanv7 [('1', 1634, 217, 1648, 251, '0.362793'), ('1', 1312, 284, 1330, 325, '0.400635'), ('1', 1586, 234, 1600, 278, '0.482178'), ('1', 862, 410, 901, 467, '0.765137'), ('1', 1592, 446, 1625, 541, '0.792969'), ('1', 1646, 332, 1670, 401, '0.873535'), ('1', 1542, 430, 1584, 539, '0.895996'), ('1', 516, 528, 560, 637, '0.918945')]
humanv7 len 8
human [('1', 1635, 216, 1649, 249, '0.442168'), ('1', 1312, 280, 1329, 323, '0.539732'), ('1', 1586, 234, 1603, 274, '0.54702'), ('1', 862, 410, 901, 467, '0.765137'), ('1', 1592, 446, 1625, 541, '0.792969'), ('1', 1646, 332, 1670, 401, '0.873535'), ('1', 1542, 430, 1584, 539, '0.895996'), ('1', 516, 528, 5

26-11-2021_11-44-10_AM.jpg
humanv5 [('1', 919, 29, 1063, 317, '0.717273'), ('1', 954, 2, 1154, 305, '0.806661')]
humanv5 len 2
humanv7 [('1', 929, 12, 1143, 316, '0.806152')]
humanv7 len 1
human [('1', 919, 29, 1063, 317, '0.717273'), ('1', 954, 2, 1154, 305, '0.806661')]
human len 2
23-11-2021_05-36-00_PM.jpg
humanv5 [('1', 406, 502, 455, 613, '0.585619'), ('1', 2090, 295, 2150, 457, '0.749122')]
humanv5 len 2
humanv7 [('1', 1194, 99, 1215, 143, '0.25415'), ('1', 1128, 128, 1163, 177, '0.269043'), ('1', 2088, 294, 2149, 473, '0.577148')]
humanv7 len 3
human [('1', 2090, 295, 2150, 457, '0.749122'), ('1', 406, 502, 455, 613, '0.585619')]
human len 2
25-11-2021_06-21-32_PM.jpg
humanv5 [('1', 1133, 126, 1159, 175, '0.403908'), ('1', 1004, 98, 1027, 161, '0.504556'), ('1', 1090, 108, 1115, 175, '0.598847'), ('1', 1528, 1132, 1605, 1292, '0.725456'), ('1', 1396, 1079, 1539, 1301, '0.790374'), ('1', 1456, 282, 1542, 474, '0.88488')]
humanv5 len 6
humanv7 [('1', 1004, 100, 1030, 159, '0.2983

24-11-2021_07-27-02_PM.jpg
humanv5 [('1', 1364, 328, 1399, 413, '0.861233')]
humanv5 len 1
humanv7 [('1', 1364, 328, 1398, 415, '0.899902')]
humanv7 len 1
human [('1', 1364, 328, 1398, 415, '0.899902')]
human len 1
24-11-2021_01-55-37_AM.jpg
humanv5 [('1', 422, 286, 463, 390, '0.625146'), ('1', 242, 22, 271, 109, '0.632584')]
humanv5 len 2
humanv7 [('1', 239, 10, 273, 95, '0.689453')]
humanv7 len 1
human [('1', 239, 10, 273, 95, '0.689453'), ('1', 422, 286, 463, 390, '0.625146')]
human len 2
26-11-2021_07-46-51_AM.jpg
humanv5 [('1', 1554, 408, 1654, 570, '0.769501')]
humanv5 len 1
humanv7 [('1', 1548, 412, 1656, 560, '0.899414')]
humanv7 len 1
human [('1', 1548, 412, 1656, 560, '0.899414')]
human len 1
25-11-2021_04-53-39_PM.jpg
humanv5 [('1', 1286, 438, 1321, 517, '0.429763'), ('1', 1326, 398, 1367, 512, '0.48834'), ('1', 1442, 384, 1468, 445, '0.513531'), ('1', 1400, 410, 1433, 491, '0.524887'), ('1', 1442, 415, 1476, 503, '0.564861'), ('1', 1510, 370, 1537, 447, '0.604096'), ('1', 1

humanv5 [('1', 374, 336, 395, 377, '0.580244'), ('1', 790, 310, 821, 382, '0.742659'), ('1', 1403, 366, 1441, 476, '0.842802')]
humanv5 len 3
humanv7 [('1', 792, 314, 827, 372, '0.268066'), ('1', 375, 338, 393, 383, '0.400635'), ('1', 1403, 364, 1441, 478, '0.900879')]
humanv7 len 3
human [('1', 790, 310, 821, 382, '0.742659'), ('1', 374, 336, 395, 377, '0.580244'), ('1', 1403, 364, 1441, 478, '0.900879')]
human len 3
26-11-2021_12-07-24_PM.jpg
humanv5 [('1', 676, 234, 725, 347, '0.81344'), ('1', 584, 230, 629, 348, '0.862326')]
humanv5 len 2
humanv7 [('1', 675, 234, 727, 351, '0.883789'), ('1', 584, 230, 627, 351, '0.931152')]
humanv7 len 2
human [('1', 675, 234, 727, 351, '0.883789'), ('1', 584, 230, 627, 351, '0.931152')]
human len 2
23-11-2021_10-40-20_PM.jpg
humanv5 [('1', 426, 236, 458, 315, '0.540555')]
humanv5 len 1
humanv7 []
humanv7 len 0
human []
human len 0
26-11-2021_12-30-08_PM.jpg
humanv5 [('1', 890, 528, 905, 559, '0.346372'), ('1', 124, 847, 221, 927, '0.404484'), ('1'

humanv5 [('1', 1031, 414, 1101, 596, '0.913097')]
humanv5 len 1
humanv7 [('1', 1032, 417, 1101, 597, '0.868164')]
humanv7 len 1
human [('1', 1031, 414, 1101, 596, '0.913097')]
human len 1
24-11-2021_09-21-31_AM.jpg
humanv5 [('1', 416, 519, 460, 625, '0.57591'), ('1', 1742, 322, 1799, 478, '0.681964'), ('1', 502, 580, 559, 741, '0.708466'), ('1', 1608, 300, 1679, 480, '0.743985')]
humanv5 len 4
humanv7 [('1', 1746, 332, 1801, 467, '0.548828'), ('1', 1612, 308, 1685, 495, '0.760742')]
humanv7 len 2
human [('1', 1742, 322, 1799, 478, '0.681964'), ('1', 1612, 308, 1685, 495, '0.760742'), ('1', 416, 519, 460, 625, '0.57591'), ('1', 502, 580, 559, 741, '0.708466')]
human len 4
24-11-2021_03-06-50_AM.jpg
humanv5 [('1', 804, 720, 928, 1057, '0.869173')]
humanv5 len 1
humanv7 [('1', 1062, 416, 1122, 523, '0.310791'), ('1', 796, 717, 937, 1053, '0.902832')]
humanv7 len 2
human [('1', 796, 717, 937, 1053, '0.902832')]
human len 1
24-11-2021_11-44-39_AM.jpg
humanv5 [('1', 1438, 120, 1470, 197, '0.

# Объединение детекций между yolov7 и yolov7-e6e для классов car, human

In [252]:
path_labelv5 = '../yolov7/runs/detect/exp_car_human_res_concat/labels/'
path_labelv7 = '../yolov7/runs/detect/exp_car_human_yolov7_e6e/labels/'

path_v7_res  = '../yolov7/runs/detect/exp_car_human_res_concat_e/'

path_images  = '../yolov7/runs/detect/exp_car_human/'

save_dir = Path(Path(path_v7_res) / 'labels')
save_dir.mkdir(parents=True, exist_ok=True)


for image_file in os.listdir(path_images):
    print(image_file)
    
#     if image_file != "23-11-2021_02-37-35_PM.jpg":
#             continue
    
    t = magic.from_file(path_images + image_file)
    w, h = t.split(', ')[-2].split('x')
    
    label_file = image_file[:-3] + 'txt'
    
    image = cv2.imread(path_images + image_file)
    
    labelv5_path = path_labelv5 + label_file
    labelv7_path = path_labelv7 + label_file
    
    existsv5 = exists(labelv5_path)
    existsv7 = exists(labelv7_path)
    
#     print(existsv5, existsv7)
    
    if existsv5 and existsv7:
        carv5, humanv5 = read_label(labelv5_path)
        carv7, humanv7 = read_label_yolo(labelv7_path)
    
    elif not existsv5 and existsv7:
        shutil.copy(labelv7_path, labelv5_path)
        carv5, humanv5 = read_label_yolo(labelv5_path)
        carv7, humanv7 = read_label_yolo(labelv7_path)
        
    elif not existsv7 and existsv5:
        shutil.copy(labelv5_path, labelv7_path)
        carv5, humanv5 = read_label(labelv5_path)
        carv7, humanv7 = read_label(labelv7_path)
    
#     print(humanv5)
#     print(humanv7)
    humanv5 = [yolo2bb(line, int(w), int(h)) for line in humanv5]
    humanv7 = [yolo2bb(line, int(w), int(h)) for line in humanv7]
#     print("humanv7", humanv5)
#     print("humanv7 len", len(humanv5))
#     print("humanv7e", humanv7)
#     print("humanv7e len", len(humanv7))
    
    # Human processing
    human = []
    for detv7 in humanv7:
#         print(detv7)
        processed = False
        ious = []
#         print("humanv5 <<<<<", humanv5)
        for detv5 in humanv5:
            ious.append((bb_intersection_over_union(detv7[1:5], detv5[1:5]), detv5))
#         print("ious", ious)
        
        ioucount = [x for x in ious if x[0] > 0.50]
#         print(len(ioucount))
        if len(ioucount) > 1:
            for iou in ious:
                human.append(iou[1])
                humanv5.remove(iou[1])
                processed = True
        else:
            for iou in ious:
                if iou[0] > 0.80:
                    tofile = detv7 if detv7[5] > iou[1][5] else iou[1]
                    human.append(tofile)
                    humanv5.remove(iou[1])
                    processed = True
                elif iou[0] < 0.80 and iou[0] > 0.50 and \
                (detv7[3]-detv7[1])*(detv7[4]-detv7[2])*100/ (int(w)*int(h)) < 1:
                    tofile = detv7 if detv7[5] > iou[1][5] else iou[1]
                    human.append(tofile)
                    humanv5.remove(iou[1])
                    processed = True
                
        if not processed:
            if float(detv7[5]) >= 0.45:
                    human.append(detv7) 
    
    if len(humanv5) != 0:
        for detv5 in humanv5:
            if float(detv5[5]) > 0.50:
                human.append(detv5)
                
    
    pairs = [(x,y, bb_intersection_over_union(x[1:5], y[1:5])) for x in human for y in human if x != y]
    iou_pairs = [x for x in pairs if x[2] > 0.68]
    iou_pairs = iou_pairs[: int( len(iou_pairs)/2 )]
#     print(len(pairs), len(iou_pairs))
#     print(iou_pairs)
    for iou_p in iou_pairs:
        if iou_p[0][5] > iou_p[1][5]:
            if iou_p[1] in human:
                human.remove(iou_p[1])
        elif iou_p[0][5] < iou_p[1][5]:
            if iou_p[0] in human:
                human.remove(iou_p[0])
        
                
#     print("human", human)
#     print("human len", len(human))
    
    
    
    
    
    carv5 = [yolo2bb(line, int(w), int(h)) for line in carv5]
    carv7 = [yolo2bb(line, int(w), int(h)) for line in carv7]
#     print("carv7", carv5)
#     print("carv7 len", len(carv5))
#     print("carv7e", carv7)
#     print("carv7e len", len(carv7))
    
    
    # Car processing
    car = []
    for detv7 in carv7:
#         print(detv7)
        processed = False
        ious = []
#         print("carv5 <<<<<", carv5)
        for detv5 in carv5:
            ious.append((bb_intersection_over_union(detv7[1:5], detv5[1:5]), detv5))
#         print("ious", ious)
        
        ioucount = [x for x in ious if x[0] > 0.50]
#         print(len(ioucount))
        if len(ioucount) > 1:
            for iou in ious:
                car.append(iou[1])
                carv5.remove(iou[1])
                processed = True
        else:
            for iou in ious:
                if iou[0] > 0.80:
                    tofile = detv7 if detv7[5] > iou[1][5] else iou[1]
                    car.append(tofile)
                    carv5.remove(iou[1])
                    processed = True
                elif iou[0] < 0.80 and iou[0] > 0.50 and \
                (detv7[3]-detv7[1])*(detv7[4]-detv7[2])*100/ (int(w)*int(h)) < 1:
                    tofile = detv7 if detv7[5] > iou[1][5] else iou[1]
                    car.append(tofile)
                    carv5.remove(iou[1])
                    processed = True
                
        if not processed:
            if float(detv7[5]) >= 0.51:
                    car.append(detv7) 
    
    if len(carv5) != 0:
        for detv5 in carv5:
            if float(detv5[5]) > 0.50:
                car.append(detv5)
                
    
    pairs = [(x,y, bb_intersection_over_union(x[1:5], y[1:5])) for x in car for y in car if x != y]
    iou_pairs = [x for x in pairs if x[2] > 0.68]
    iou_pairs = iou_pairs[: int( len(iou_pairs)/2 )]
#     print(len(pairs), len(iou_pairs))
#     print(iou_pairs)

    for iou_p in iou_pairs:
        if iou_p[0][5] > iou_p[1][5]:
            if iou_p[1] in car:
                car.remove(iou_p[1])
        elif iou_p[0][5] < iou_p[1][5]:
#             print(iou_p)
            if iou_p[0] in car:
                car.remove(iou_p[0])
        
                
#     print("car", car)
#     print("car len", len(car))
    
    
    
    

    with open(path_v7_res + '/labels/' + label_file, 'w') as f:
        for hum_lbl in human:
#             print(hum_lbl)
            
            box = ( int(hum_lbl[1]), int(hum_lbl[2]), int(hum_lbl[3]), int(hum_lbl[4])  )
            
            draw_label(hum_lbl[0], box, image)
            
            lbl = bb_to_yolo(box, int(w), int(h))
            data = hum_lbl[0] + ' ' + ' '.join(str(e) for e in lbl) + ' ' + hum_lbl[5] + '\n'
            f.write(data)
            
        for car_lbl in car:
#             print(car_lbl)
            
#             car_lbl = yolo2bb(car_l, int(w), int(h))
            
            box = ( int(car_lbl[1]), int(car_lbl[2]), int(car_lbl[3]), int(car_lbl[4])  )
            
            draw_label(car_lbl[0], box, image)
            
            lbl = bb_to_yolo(box, int(w), int(h))
            data = car_lbl[0] + ' ' + ' '.join(str(e) for e in lbl) + ' ' + car_lbl[5] + '\n'
            
            f.write(data)
            
    cv2.imwrite(path_v7_res + '/' + image_file, image)
            
    
#     break

26-11-2021_08-22-27_AM.jpg
25-11-2021_01-55-28_PM.jpg
25-11-2021_10-02-36_AM.jpg
23-11-2021_07-41-17_PM.jpg
24-11-2021_01-31-29_PM.jpg
23-11-2021_05-01-08_PM.jpg
25-11-2021_03-59-27_PM.jpg
25-11-2021_03-22-37_PM.jpg
24-11-2021_04-36-03_AM.jpg
24-11-2021_01-31-40_PM.jpg
24-11-2021_06-40-20_AM.jpg
25-11-2021_08-44-40_PM.jpg
24-11-2021_08-28-27_AM.jpg
26-11-2021_08-24-11_AM.jpg
25-11-2021_06-44-40_AM.jpg
24-11-2021_02-43-32_PM.jpg
26-11-2021_12-27-04_PM.jpg
24-11-2021_07-53-17_AM.jpg
24-11-2021_05-48-42_AM.jpg
26-11-2021_07-47-56_AM.jpg
23-11-2021_03-14-14_PM.jpg
24-11-2021_04-53-50_AM.jpg
26-11-2021_08-58-00_AM.jpg
25-11-2021_01-37-49_PM.jpg
24-11-2021_06-23-42_AM.jpg
23-11-2021_03-49-17_PM.jpg
24-11-2021_09-02-57_AM.jpg
24-11-2021_06-33-29_PM.jpg
24-11-2021_06-41-22_AM.jpg
25-11-2021_10-56-11_AM.jpg
26-11-2021_04-48-23_AM.jpg
25-11-2021_05-51-37_AM.jpg
26-11-2021_10-09-10_AM.jpg
25-11-2021_10-56-23_AM.jpg
25-11-2021_02-13-20_PM.jpg
25-11-2021_06-04-02_PM.jpg
26-11-2021_04-29-13_AM.jpg
2

24-11-2021_12-19-34_PM.jpg
24-11-2021_04-18-41_AM.jpg
24-11-2021_02-06-04_PM.jpg
25-11-2021_09-09-37_AM.jpg
26-11-2021_06-16-35_AM.jpg
25-11-2021_02-14-20_PM.jpg
24-11-2021_04-48-35_PM.jpg
23-11-2021_02-56-22_PM.jpg
26-11-2021_08-05-04_AM.jpg
23-11-2021_04-25-46_PM.jpg
24-11-2021_06-23-20_AM.jpg
24-11-2021_02-31-12_AM.jpg
24-11-2021_10-16-37_AM.jpg
24-11-2021_09-05-17_AM.jpg
24-11-2021_09-33-08_PM.jpg
24-11-2021_11-25-22_AM.jpg
25-11-2021_10-21-41_AM.jpg
25-11-2021_01-56-10_PM.jpg
23-11-2021_06-46-48_PM.jpg
23-11-2021_02-37-22_PM.jpg
24-11-2021_03-17-57_PM.jpg
24-11-2021_03-37-19_PM.jpg
24-11-2021_12-03-08_PM.jpg
24-11-2021_10-33-07_AM.jpg
25-11-2021_03-06-22_PM.jpg
24-11-2021_02-43-15_PM.jpg
26-11-2021_03-35-26_AM.jpg
24-11-2021_03-42-06_AM.jpg
25-11-2021_01-37-43_PM.jpg
26-11-2021_12-04-22_PM.jpg
24-11-2021_07-53-46_AM.jpg
25-11-2021_07-34-00_PM.jpg
26-11-2021_08-05-21_AM.jpg
24-11-2021_08-46-26_AM.jpg
24-11-2021_09-59-05_AM.jpg
25-11-2021_06-09-42_AM.jpg
26-11-2021_12-37-22_AM.jpg
2

24-11-2021_09-57-05_AM.jpg
25-11-2021_05-47-16_PM.jpg
24-11-2021_03-19-10_PM.jpg
25-11-2021_03-23-37_PM.jpg
24-11-2021_03-36-34_PM.jpg
26-11-2021_09-33-08_AM.jpg
24-11-2021_06-59-50_AM.jpg
26-11-2021_03-35-23_AM.jpg
25-11-2021_06-22-44_PM.jpg
25-11-2021_04-52-29_PM.jpg
26-11-2021_06-34-15_AM.jpg
24-11-2021_10-50-59_AM.jpg
24-11-2021_07-09-36_PM.jpg
24-11-2021_04-11-15_PM.jpg
25-11-2021_06-03-41_PM.jpg
26-11-2021_03-53-42_AM.jpg
25-11-2021_08-14-18_AM.jpg
24-11-2021_09-04-10_AM.jpg
25-11-2021_10-20-40_AM.jpg
26-11-2021_09-52-01_AM.jpg
24-11-2021_05-23-52_PM.jpg
26-11-2021_07-27-54_AM.jpg
26-11-2021_04-48-11_AM.jpg
24-11-2021_07-00-36_AM.jpg
25-11-2021_10-38-55_AM.jpg
26-11-2021_11-45-44_AM.jpg
24-11-2021_04-01-33_AM.jpg
26-11-2021_03-53-37_AM.jpg
24-11-2021_06-24-38_AM.jpg
24-11-2021_01-12-19_PM.jpg
26-11-2021_06-52-37_AM.jpg
25-11-2021_06-39-38_PM.jpg
24-11-2021_10-32-39_AM.jpg
25-11-2021_10-39-17_AM.jpg
25-11-2021_02-31-38_PM.jpg
25-11-2021_03-30-08_AM.jpg
24-11-2021_02-42-12_PM.jpg
2